In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from childes_mi.utils.paths import DATA_DIR, FIGURE_DIR, ensure_dir
from childes_mi.utils.general import flatten,save_fig

In [3]:
from childes_mi.information_theory import model_fitting as mf

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
MI_DF = pd.read_pickle(DATA_DIR / "mi" / "phonbank_mi_full_median_shuffled.pickle")

In [6]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_phones,n_unique_phones,median_n_phones,shuffling
0,"[10.462783652449525, 9.983388971328182, 9.7520...","[0.0049862970087629875, 0.004861915778448901, ...","[9.581032058713163, 9.580828108367228, 9.58096...","[0.004776055346142387, 0.004775665750301762, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3,20,310065,0,1574,full
1,"[5.740400393887644, 5.659412386387666, 5.62936...","[0.009623133671282348, 0.009617792466309692, 0...","[5.60940114322489, 5.609992145894537, 5.609532...","[0.009598316995692623, 0.009589218644341833, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3,20,310065,0,1574,shuffled_within_words
2,"[10.41829005815009, 9.936921844638222, 9.69985...","[0.004978099712851141, 0.00484636226919765, 0....","[9.579728969514637, 9.581403782018807, 9.57992...","[0.004774484282599742, 0.004775984491959822, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3,20,310065,0,1574,shuffled_between_words


In [7]:
MI_DF.shuffling.unique()

array(['full', 'shuffled_within_words', 'shuffled_between_words',
       'shuffled_within_utterance', 'no_repeats',
       'shuffled_between_utterance', 'shuffled_within_transcript'],
      dtype=object)

In [8]:
MI_DF = MI_DF.assign(**{i:np.nan for i in ['exp_results', 'pow_results', 'concat_results',
     'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
     'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak']})
MI_DF['curvature'] = MI_DF['curvature'].astype(object)

In [9]:
n = 100 # max distance for computation
for idx, row in tqdm(MI_DF.iterrows(), total=len(MI_DF)):
    # get signal
    sig = np.array(row.MI-row.shuff_MI)
    distances = row.distances
    sig = sig
    
    dist_mask = distances <= row.median_n_phones
    distances = distances[dist_mask]
    sig = sig[dist_mask]
    
    # fit models
    results_power, results_exp, results_pow_exp, best_fit_model = mf.fit_models(distances, sig)
    
    # get fit results
    R2_exp, R2_concat, R2_power, AICc_exp, \
        AICc_pow, AICc_concat = mf.fit_results(sig, distances, 
                                              results_exp, results_power,
                                              results_pow_exp)
    
    
    # get model y
    distances_mod = np.logspace(0,np.log10(n), base=10, num=1000)
    if best_fit_model == 'pow_exp':
        y_model = mf.get_y(mf.pow_exp_decay, results_pow_exp, distances_mod)
    elif best_fit_model == 'exp':
        y_model = mf.get_y(mf.exp_decay, results_exp, distances_mod)
    elif best_fit_model == 'pow':
        y_model = mf.get_y(mf.powerlaw_decay, results_power, distances_mod)
    
    # get curvature of model_y
    curvature_model = mf.curvature(np.log(y_model))
    
    # if the best fit model is pow_exp, then grab the min peak
    if best_fit_model == 'pow_exp':
        # get peaks of curvature
        peaks = np.where((
            (curvature_model[:-1] < curvature_model[1:])[1:] & (curvature_model[1:] < curvature_model[:-1])[:-1]
        ))
        min_peak = peaks[0][0]
    else:
        min_peak = np.nan

    # get save model fit results to MI_DF
    MI_DF.loc[idx, np.array(['exp_results', 'pow_results', 'concat_results',
                         'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
                         'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak'])] = [
        results_exp, results_power, results_pow_exp,
        R2_exp, R2_concat, R2_power, AICc_exp,
        AICc_concat, AICc_pow, best_fit_model,
        curvature_model, min_peak
    ]

    print(row.age_cohort_low, row.age_cohort_high, best_fit_model, row.shuffling)

/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/childes_mi/information_theory/model_fitting.py:8: RuntimeWarning: invalid value encountered in log
  return np.abs(np.log(y_true) - np.log(y_model)) * (1 / (np.log(1 + x)))


3 20 pow_exp full
3 20 pow_exp shuffled_within_words
3 20 pow_exp shuffled_between_words
3 20 pow_exp shuffled_within_utterance
3 20 pow_exp no_repeats
3 20 pow_exp shuffled_between_utterance
3 20 pow shuffled_within_transcript
2.5 3 pow_exp full
2.5 3 pow_exp shuffled_within_words
2.5 3 exp shuffled_between_words
2.5 3 pow_exp shuffled_within_utterance
2.5 3 pow_exp no_repeats
2.5 3 pow_exp shuffled_between_utterance
2.5 3 pow shuffled_within_transcript
2 2.5 pow_exp full
2 2.5 pow shuffled_within_words
2 2.5 exp shuffled_between_words
2 2.5 pow_exp shuffled_within_utterance
2 2.5 pow_exp no_repeats
2 2.5 pow_exp shuffled_between_utterance
2 2.5 exp shuffled_within_transcript
1.5 2 pow_exp full
1.5 2 pow_exp shuffled_within_words
1.5 2 exp shuffled_between_words
1.5 2 pow_exp shuffled_within_utterance
1.5 2 pow_exp no_repeats
1.5 2 pow_exp shuffled_between_utterance
1.5 2 pow shuffled_within_transcript
1 1.5 pow_exp full
1 1.5 pow_exp shuffled_within_words
1 1.5 pow_exp shuffled_betwe

In [10]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_phones,n_unique_phones,median_n_phones,...,concat_results,R2_exp,R2_concat,R2_power,AICc_exp,AICc_concat,AICc_power,bestfitmodel,curvature,min_peak
0,"[10.462783652449525, 9.983388971328182, 9.7520...","[0.0049862970087629875, 0.004861915778448901, ...","[9.581032058713163, 9.580828108367228, 9.58096...","[0.004776055346142387, 0.004775665750301762, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3,20,310065,0,1574,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.803401,0.841454,0.840546,-5687.125636,-5998.032033,-5993.698165,pow_exp,"[-3.466673961285433e-06, -5.218923875438178e-0...",189.0
1,"[5.740400393887644, 5.659412386387666, 5.62936...","[0.009623133671282348, 0.009617792466309692, 0...","[5.60940114322489, 5.609992145894537, 5.609532...","[0.009598316995692623, 0.009589218644341833, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3,20,310065,0,1574,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.744447,0.806274,0.804592,-3315.562404,-3589.895654,-3585.247737,pow_exp,"[-7.665690649704763e-06, -1.1500075645919643e-...",6.0
2,"[10.41829005815009, 9.936921844638222, 9.69985...","[0.004978099712851141, 0.00484636226919765, 0....","[9.579728969514637, 9.581403782018807, 9.57992...","[0.004774484282599742, 0.004775984491959822, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3,20,310065,0,1574,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.857692,0.858956,0.850476,-3204.513272,-3208.395193,-3160.587539,pow_exp,"[-1.0032263994535291e-05, -1.5071595313980539e...",316.0


In [11]:
MI_DF.columns

Index(['MI', 'MI_var', 'shuff_MI', 'shuff_MI_var', 'distances',
       'age_cohort_low', 'age_cohort_high', 'n_phones', 'n_unique_phones',
       'median_n_phones', 'shuffling', 'exp_results', 'pow_results',
       'concat_results', 'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
       'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak'],
      dtype='object')

In [12]:
parameter_translation = {
    "e_init": "a",
    "e_decay_const": "b",
    "p_init": "c",
    "p_decay_const": "d",
    "intercept": "f",
}

In [13]:
param_df = pd.DataFrame(
    columns=[
        "dataset", 
        "age_low", 
        "age_high", 
        "a_value", 
        "a_stderr",
        "b_value", 
        "b_stderr",
        "c_value", 
        "c_stderr",
        "d_value", 
        "d_stderr",
        "f_value", 
        "f_stderr"
    ]
)
for idx, row in MI_DF[MI_DF.shuffling == 'full'].iterrows():
    param_vals = {}
    param_stderrs = {}
    for param in row.concat_results.params:
        param_vals[param] = row.concat_results.params[param].value
        param_stderrs[param] = row.concat_results.params[param].stderr
    param_df.loc[len(param_df)] = [
        'phonbank',
        row.age_cohort_low,
        row.age_cohort_high,
        param_vals["e_init"],
        param_stderrs["e_init"],
        param_vals["e_decay_const"],
        param_stderrs["e_decay_const"],
        param_vals["p_init"],
        param_stderrs["p_init"],
        param_vals["p_decay_const"],
        param_stderrs["p_decay_const"],
        param_vals["intercept"],
        param_stderrs["intercept"],

    ]

In [14]:
param_df

,dataset,age_low,age_high,a_value,a_stderr,b_value,b_stderr,c_value,c_stderr,d_value,d_stderr,f_value,f_stderr
0,phonbank,3,20,1.019228,0.371470,0.856513,0.136712,0.475605,0.132080,-1.432611,0.087300,0.001397,0.000040
1,phonbank,2.5,3,1.112392,0.254874,0.907804,0.100338,0.379824,0.082137,-1.380756,0.069510,0.001498,0.000038
2,phonbank,2,2.5,1.224931,0.136307,0.877363,0.054054,0.304584,0.043078,-1.219118,0.046071,0.002378,0.000049
3,phonbank,1.5,2,0.891221,0.098149,0.794046,0.032376,0.358147,0.041934,-1.234150,0.044388,0.005142,0.000117
4,phonbank,1,1.5,0.404213,0.046688,0.463113,0.021292,0.446015,0.028507,-1.136704,0.027038,0.016470,0.000233
5,phonbank,0.5,1,0.325817,0.065498,0.390629,0.045000,0.301216,0.040821,-1.013226,0.087136,0.035119,0.001961


In [15]:
ensure_dir(DATA_DIR / 'param_dfs')
param_df.to_pickle(DATA_DIR / 'param_dfs' / 'phonbank.pickle')